In [134]:
%pip install requests 

Note: you may need to restart the kernel to use updated packages.


In [135]:
import requests
from bs4 import BeautifulSoup

url = 'https://mitmachen.siegburg.de/angebotslandkarte'

response = requests.get(url)

links = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    a_tags = soup.find_all('a')

    for tag in a_tags:
        href = tag.get('href')
        
        if href and '/proposals/' in href and not 'new' in href:
            link = 'https://mitmachen.siegburg.de' + href
    
            if not link in links:
                links.append(link)
    
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


In [136]:
import pandas as pd

In [137]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Updated function to extract proposals from a project page
def extract_proposals(soup, base_url):
    proposals = []
    proposal_items = soup.find_all('div', class_='resource-item proposal-list-item')

    for proposal in proposal_items:
        # Extract title
        title_tag = proposal.find('a', class_='resource-item--title')
        title = title_tag.get_text(strip=True) if title_tag else None

        # Extract URL
        url = base_url + title_tag['href'] if title_tag and 'href' in title_tag.attrs else None

        # Extract description
        description_tag = proposal.find('div', class_='resource-item--description')
        description = description_tag.get_text(strip=True) if description_tag else None

        # Extract author/username
        author_tag = proposal.find('a', class_='resource-item--author')
        author = author_tag.get_text(strip=True) if author_tag else None

print(link_and_data)

{'https://mitmachen.siegburg.de/proposals/1090-senioiren-cafe-lichtblick-kaldauen': 'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*innenUnterstützung0 Unterstützer*innenUm fortzufahren, m

In [151]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(link_and_data.items()), columns=['URL', 'Content'])

df['URL'][:5]

0    https://mitmachen.siegburg.de/proposals/1090-s...
1    https://mitmachen.siegburg.de/proposals/1089-d...
2    https://mitmachen.siegburg.de/proposals/1087-g...
3    https://mitmachen.siegburg.de/proposals/1056-g...
4    https://mitmachen.siegburg.de/proposals/1055-b...
Name: URL, dtype: object

In [152]:
df.shape

(24, 2)

In [153]:
# Display the resulting DataFrame to the user
df['Content'][0]

'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*innenUnterstützung0 Unterstützer*innenUm fortzufahren, müssen Sie sichanmeldenoderregistrieren.TeilenTeilenwhatsapp'

In [154]:
df['Content'][0]

'Senioiren Cafe Lichtblick Kaldauen18. September 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte60-7475+Geselliges BeisammenseinDas Cafe Lichtblick in Kaldauen lädt alle Senioren zum geselligen Beisammensein bei Kaffee und Kuchen ein. Es wird gesungen, gespielt, getanzt, gelacht und viel erzählt. Auch gemeinsame Ausflüge und Feiern stehen auf dem Programm.Haben Sie Interesse, dann kommen Sie gerne vorbei. Die Senioren und das Team freut sich auf Sie!Treffen : Alle 14 Tage Dienstags von 15.00 - 17.00 Uhr\xa0(immer in geraden Wochen)Ort : Pfarrheim der katholischen Kirche KaldauenAntoniusweg 1 ;\xa053721 SiegburgAnmeldungen bei Rita Quadt ; Tel.:\xa0015733725174:Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.MCafé LichtblickBeigetreten am: 18.09.2024Konto verifiziertVorschläge1Unterstützer*innenUnterstützer*innenUnterstützung0 Unterstützer*innenUm fortzufahren, müssen Sie sichanmeldenoderregistrieren.TeilenTeilenwhatsapp'

In [162]:
import re

        # Extract parent project
        project_tag = proposal.find('a', class_='breadcrumbs-item')
        proposed_for_project = project_tag.get_text(strip=True) if project_tag else None

        proposals.append({
            'URL': url,
            'Title': title,
            'Proposed for Project': proposed_for_project,
            'Description': description,
            'Author': author,
            'Comments': comments,
            'Supporters': supporters,
        })
    return proposals


# Function to extract city name from the base URL
def extract_city_name(base_url):
    # Words to remove from the city name
    remove_words = ['mitmachen', 'Mitmachen', 'mitwirken', 'Smarte', 'region', 'unser', 'mitgestalten', 'gestalten', 'machmit', 'dialog', 'consul', 'www', 'de', 'https', 'com']

    # Split the URL into parts (by '.' or '/')
    parts = base_url.replace('https://', '').replace('http://', '').split('.')
    all_parts = [part.split('/')[0] for part in parts]  # Handle cases where "/" exists after domain

    # Remove known unwanted words and empty strings
    filtered_parts = [part for part in all_parts if part.lower() not in remove_words and part]

    # Return the first relevant part (assumes city name is left after filtering)
    city = filtered_parts[0].replace('-', ' ').capitalize() if filtered_parts else "Unknown"

    # Remove unwanted words from city name
    for word in remove_words:
        city = city.replace(word, '')

    return city.strip().capitalize()


# Update project scraping to exclude comments
def scrape_project_page_with_proposals(url, base_url, city):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to load project page: {url}")
        return None, []

# Apply the enhanced function to the DataFrame and create new columns
df[['Title', 'Date', 'Comments', 'Tags', 'Description', 'Username', 'Vorschläge', 'Konto Status', 'Supporters']] = df['Content'].apply(
    lambda x: pd.Series(extract_full_data_with_supporters(x))
)


In [163]:
df['Comments'].replace(20240, 0, inplace=True)

C:\Users\Rusel\AppData\Local\Temp\ipykernel_15012\2378994937.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Comments'].replace(20240, 0, inplace=True)


In [174]:
df['Content'][3]

'Geselliges Beisammensein in Braschoß18. August 20240 KommentareZugehöriges Projekt:\n            Angebotslandkarte18-2425-4950-5960-7475+Geselliges BeisammenseinDer "Pfarrverein Braschoß" lädt regelmäßig zum geselligen Beisammensein nach Braschoß ein:- Skatabende- Leseabende- Spielabend- ...Konkrete Veranstaltungen werden unterhttps://www.pfarrverein-braschoss.de/aktuelles.htmlveröffentlicht.Gerne informieren wir Sie per Mail über neue Veranstaltungen:https://www.pfarrverein-braschoss.de/newsletter.htmlAlle interessierte Bürger:innen\xa0 sind herzlich willkommen. In der Veranstaltungsübersicht ist vermerkt, wenn vorab eine Anmeldung erforderlich ist.Die Veranstaltungen finden in 53721 Siegburg-Braschoß, Braschosser Str. 50 statt.Kommentare(0)Um fortzufahren, müssen Sie sichanmeldenoderregistrieren.KEEBeigetreten am: 20.11.2022Konto verifiziertVorschläge3Unterstützer*innenUnterstützer*innenUnterstützung4 Unterstützer*innenUm fortzufahren, müssen Sie sichanmeldenoderregistrieren.Dokumen

In [171]:
df['URL'][3]

'https://mitmachen.siegburg.de/proposals/1056-geselliges-beisammensein-in-braschoss'

In [166]:
df['URL'][2]

'https://mitmachen.siegburg.de/proposals/1087-gemeinschaftsgarten-cecilienstrasse'

In [167]:
# Function to clean description considering keywords, numeric patterns, and refined starting logic
def clean_description_advanced(content):
    # Define keywords that mark the beginning of the description
    keywords = [
        'Geselliges Beisammensein', 'Natur', 'Hilfe & Beratung', 'Bildung', 
        'Musik', 'Bewegung', 'Glaube', 'Kulinarisches', 'Kunst & Kultur', 'Sonstiges',
    ]
    
    # Check for keywords first
    for keyword in keywords:
        if keyword in content:
            start_idx = content.find(keyword) + len(keyword)
            description = content[start_idx:].strip()
            description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
            return description

    # If no keyword is found, check for numeric patterns like "18-24, 25-49, etc."
    numeric_pattern = re.search(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
    if numeric_pattern:
        start_idx = numeric_pattern.end()
        description = content[start_idx:].strip()
        description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
        return description


    # If nothing works, return the content as is
    return content

# Apply the advanced cleaning function to the Description column
df['Description'] = df['Content'].apply(clean_description_advanced)


In [168]:
df

,URL,Content,Title,Date,Comments,Tags,Description,Username,Vorschläge,Konto Status,Supporters
0,https://mitmachen.siegburg.de/proposals/1090-s...,Senioiren Cafe Lichtblick Kaldauen18. Septembe...,Senioiren Cafe Lichtblick Kaldauen,18. September 2024,0,"60-74, 75+",Das Cafe Lichtblick in Kaldauen lädt alle Seni...,Senioiren Cafe,1,verifiziert,0
1,https://mitmachen.siegburg.de/proposals/1089-d...,Digitalsprechstunde für Senior*innen16. Septem...,Digitalsprechstunde für Senior*innen,16. September 2024,0,"60-74, 75+",Am ersten Donnerstag im Monat erhalten Seniori...,Digitalsprechstunde für,17,verifiziert,0
2,https://mitmachen.siegburg.de/proposals/1087-g...,Gemeinschaftsgarten Cecilienstraße12. Septembe...,Gemeinschaftsgarten Cecilienstraße,12. September 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Der Gemeinschaftsgarten ist eine Kooperation d...,Gemeinschaftsgarten Cecilienstraße12,1,ist nicht verifiziert,0
3,https://mitmachen.siegburg.de/proposals/1056-g...,Geselliges Beisammensein in Braschoß18. August...,Geselliges Beisammensein in Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",in Braschoß18. August 20240 KommentareZugehöri...,Geselliges Beisammensein,3,verifiziert,4
4,https://mitmachen.siegburg.de/proposals/1055-b...,Bildungsveranstaltungen Pfarrverein Braschoß18...,Bildungsveranstaltungen Pfarrverein Braschoß,18. August 2024,0,"18-24, 25-49, 50-59, 60-74, 75+",sveranstaltungen Pfarrverein Braschoß18. Augus...,Bildungsveranstaltungen Pfarrverein,3,verifiziert,4
5,https://mitmachen.siegburg.de/proposals/1054-o...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,Öffentlicher Bücherschrank für Kinder-/Jugend-...,15. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Stöbern und Tauschen erwünscht!Den Bücherschra...,Öffentlicher Bücherschrank,3,verifiziert,2
6,https://mitmachen.siegburg.de/proposals/1052-r...,Rikscha-Fahrten in Siegburg14. August 20240 Ko...,Rikscha-Fahrten in Siegburg,14. August 2024,0,75+,Ausfahrten mit der Fahrrad Rikscha in die nähe...,Fahrten in,2,ist nicht verifiziert,4
7,https://mitmachen.siegburg.de/proposals/1051-m...,Mach was freiwillig!\r\n\r\nEhrenamt und Engag...,Mach was freiwillig!,12. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+, ...",Die Freiwilligen-Agentur für den Rhein-Sieg-Kr...,Mach was,1,ist nicht verifiziert,2
8,https://mitmachen.siegburg.de/proposals/1049-g...,Generationen-Parcours08. August 20240 Kommenta...,Generationen-Parcours,08. August 2024,0,"0-12, 13-17, 18-24, 25-49, 50-59, 60-74, 75+",Die Outdoor Fitness Anlage am Michaelsberg unt...,August 20240,17,verifiziert,0
9,https://mitmachen.siegburg.de/proposals/1041-e...,Erlebe Abenteuer mit den Annopfadfinder*innen!...,Erlebe Abenteuer mit den Annopfadfinder*innen!,17. Juli 2024,0,"0-12, 13-17, 18-24",Wir sind diePfadfinder*innen vom Stamm St. Ann...,Erlebe Abenteuer,1,verifiziert,1


In [146]:
df.to_csv('siegburg_data.csv', index=False)

In [147]:
df['URL'][14]

'https://mitmachen.siegburg.de/proposals/1030-sommerferien-workshop-self-defense-4-girls'